In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def unet_with_vgg16_backbone(input_shape=(128, 128, 3), num_classes=13):
    # Load VGG16 as the encoder
    vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    vgg16_base.trainable = False  # Freeze the encoder layers

    # Encoder - Use VGG16 layers for downsampling
    c1 = vgg16_base.get_layer("block1_conv2").output
    c2 = vgg16_base.get_layer("block2_conv2").output
    c3 = vgg16_base.get_layer("block3_conv3").output
    c4 = vgg16_base.get_layer("block4_conv3").output
    c5 = vgg16_base.get_layer("block5_conv3").output

    # Decoder - Upsampling Path
    u1 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
    u1 = layers.concatenate([u1, c4])
    u1 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u1)
    u1 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u1)

    u2 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(u1)
    u2 = layers.concatenate([u2, c3])
    u2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u2)
    u2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u2)

    u3 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(u2)
    u3 = layers.concatenate([u3, c2])
    u3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u3)
    u3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u3)

    u4 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(u3)
    u4 = layers.concatenate([u4, c1])
    u4 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u4)
    u4 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u4)

    # Output Layer
    outputs = layers.Conv2D(num_classes, (1, 1), activation='softmax')(u4)

    # Define the model
    model = models.Model(inputs=vgg16_base.input, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Build and summarize the model
vgg16_unet_model = unet_with_vgg16_backbone()
vgg16_unet_model.summary()


In [ ]:
def unet_with_resnet50_backbone(input_shape=(128, 128, 3), num_classes=3):
    # Load ResNet50 as the encoder
    resnet50_base = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    resnet50_base.trainable = False  # Freeze the encoder layers

    # Encoder - Use ResNet50 layers for downsampling
    c1 = resnet50_base.get_layer("conv1_relu").output
    c2 = resnet50_base.get_layer("conv2_block3_out").output
    c3 = resnet50_base.get_layer("conv3_block4_out").output
    c4 = resnet50_base.get_layer("conv4_block6_out").output
    c5 = resnet50_base.get_layer("conv5_block3_out").output

    # Decoder - Upsampling Path
    u1 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
    u1 = layers.concatenate([u1, c4])
    u1 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u1)
    u1 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u1)

    u2 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(u1)
    u2 = layers.concatenate([u2, c3])
    u2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u2)
    u2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u2)

    u3 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(u2)
    u3 = layers.concatenate([u3, c2])
    u3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u3)
    u3 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u3)

    u4 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(u3)
    u4 = layers.concatenate([u4, c1])
    u4 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u4)
    u4 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u4)

    # Output Layer
    outputs = layers.Conv2D(num_classes, (1, 1), activation='softmax')(u4)

    # Define the model
    model = models.Model(inputs=resnet50_base.input, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Build and summarize the model
resnet50_unet_model = unet_with_resnet50_backbone()
resnet50_unet_model.summary()


In [ ]:
# Define paths to images and masks
train_image_path = 'path_to_train_images'
train_mask_path = 'path_to_train_masks'
val_image_path = 'path_to_val_images'
val_mask_path = 'path_to_val_masks'

# Create data generators
train_image_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_image_path, target_size=(128, 128), class_mode=None, batch_size=32, seed=1)

train_mask_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_mask_path, target_size=(128, 128), class_mode=None, batch_size=32, seed=1)

val_image_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_image_path, target_size=(128, 128), class_mode=None, batch_size=32, seed=1)

val_mask_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    val_mask_path, target_size=(128, 128), class_mode=None, batch_size=32, seed=1)

# Combine generators for input images and masks
train_gen = zip(train_image_gen, train_mask_gen)
val_gen = zip(val_image_gen, val_mask_gen)

# Train the model
history = vgg16_unet_model.fit(train_gen, validation_data=val_gen, epochs=50, steps_per_epoch=100, validation_steps=25)
